In [ ]:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>
PREFIX iprok: <http://w3id.org/IproK#>

SELECT ?buildingElement ?name ?locatedIn ?locatedIn ?netVolume ?assignedTask
WHERE {
  ?buildingElement bimonto:GlobalId "2FUQJjpVj9wBkoUxY59XRk" ;
                   bimonto:Name ?name ;
                   bimonto:hasBuildingStorey ?locatedIn ;
                   bimonto:NetVolume ?netVolume ;
                   bimonto:AssignToProcess ?assignedTask .
} 

In [ ]:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>

SELECT ?buildingStorey ?buildingElement ?elementType 
WHERE {
  ?buildingStorey a bimonto:BuildingStorey;
                    bimonto:hasBuildingElement ?element .
  ?element          bimonto:Name ?buildingElement ;
                    bimonto:hasPredefinedType ?elementType .
} 

In [ ]:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>

SELECT ?building_Storey (SUM(?Net_volume) AS ?Total_NetVolume)
WHERE {
  ?building_Storey a bimonto:BuildingStorey ;
                   bimonto:hasBuildingElement ?element .
  ?element a bimonto:Wall ;
           bimonto:NetVolume ?Net_volume .
}
GROUP BY ?building_Storey

In [ ]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import URLError

def run_sparql_query(building_element_guid):
    """
    Queries the ontology for all data related to a specific building element GUID.
    """
    sparql = SPARQLWrapper("http://localhost:3030/bimontoDB/sparql")
    
    # This comprehensive query fetches all related task, time, resource, and cost data.
    # OPTIONAL blocks are used because a task may not have all types of data.
    query = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>
        PREFIX iprok: <http://w3id.org/IproK#>

        SELECT *
        WHERE {{
          ?building_element bimonto:GlobalId "{building_element_guid}" ;
                            bimonto:AssignToProcess ?task .
          
          ?task iprok:Name ?TaskName .
          
          OPTIONAL {{
            ?task iprok:hasTaskTime ?task_time .
            ?task_time iprok:ScheduleStart ?ScheduleStart ;
                       iprok:ScheduleFinish ?ScheduleFinish ;
                       iprok:ScheduleDuration ?ScheduleDuration ;
                       iprok:ActualStart ?ActualStart ;
                       iprok:ActualFinish ?ActualFinish ;
                       iprok:ActualDuration ?ActualDuration .
          }}
          
          OPTIONAL {{
            ?task iprok:requiresResource ?resource .
            ?resource iprok:Name ?ResourceName ;
                      rdf:type ?ResourceType ;
                      iprok:BudgetedUnits ?BudgetedUnits ;
                      iprok:ActualUnits ?ActualUnits ;
                      iprok:UnitPrice ?UnitPrice .
          }}
          
          OPTIONAL {{
            ?task iprok:hasCostItem ?cost_item ;
                  iprok:ActualCost ?actual_cost ;
                  iprok:BudgetedCost ?planned_cost ;
                  iprok:hasCostType ?cost_type .
                   
          }}
        }}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        print("Querying SPARQL endpoint...")
        results = sparql.query().convert()
        print("Query successful.")
        return results["results"]["bindings"]
    except URLError as e:
        print(f"Error connecting to SPARQL endpoint: {e}")
    except Exception as e:
        print(f"An error occurred during SPARQL query: {e}")
    return None